# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234722159584                   -0.50    7.0
  2   -7.250167355596       -1.81       -1.39    1.0
  3   -7.251261156371       -2.96       -2.04    3.0
  4   -7.251154301641   +   -3.97       -2.02    4.0
  5   -7.251326593294       -3.76       -2.61    4.0
  6   -7.251337338147       -4.97       -3.02    3.0
  7   -7.251338765163       -5.85       -3.97    2.0
  8   -7.251338789058       -7.62       -4.06    4.0
  9   -7.251338797722       -8.06       -4.50    2.0
 10   -7.251338798043       -9.49       -4.63    1.0
 11   -7.251338798701       -9.18       -5.63    3.0
 12   -7.251338798704      -11.44       -5.96    2.0
 13   -7.251338798704   +  -12.77       -6.15    1.0
 14   -7.251338798704      -12.54       -6.58    1.0
 15   -7.251338798705      -13.19       -7.30    3.0
 16   -7.251338798705      -14.57       -7.60    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06191881547588871
[ Info: Arnoldi iteration step 2: normres = 0.6879546438803961
[ Info: Arnoldi iteration step 3: normres = 0.7225037762415915
[ Info: Arnoldi iteration step 4: normres = 0.28602640883011976
[ Info: Arnoldi iteration step 5: normres = 0.2977348973059923
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.16e-02, 5.29e-02, 2.20e-01, 1.91e-01, 3.15e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4691845146076329
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.72e-03, 1.87e-01, 4.06e-01, 9.86e-02, 4.63e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09410062709159353
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.41e-04, 3.28e-02, 1.43e-02, 3.65e-02, 7.06e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12124274939988207
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.26e-